In [1]:
### Q1.
### Load packages and preprocess data using professor's Unit commitment problem Starter

## Load packages
using JSON
using JuMP, HiGHS

## Define function for solving Unit commitment proplem
function solve_uc(data_file, option)
    # Preprocess data
    data = JSON.parsefile(data_file)

    T = data["time_periods"]
    thermal_gens = keys(data["thermal_generators"])
    renewable_gens = keys(data["renewable_generators"])
    periods = 1:T

    demand = [data["demand"][t] for t in 1:T]

    min_down_time=Dict()
    min_up_time=Dict()
    min_power=Dict()
    max_power=Dict()
    unit_cost=Dict()
    for (g,gen) in data["thermal_generators"]
        min_down_time[g]= gen["time_down_minimum"]
        min_up_time[g]=gen["time_up_minimum"]
        min_power[g]=gen["power_output_minimum"]
        max_power[g]=gen["power_output_maximum"]
        unit_cost[g]=gen["piecewise_production"][1]["cost"]
    end

    rgen_min=Dict()
    rgen_max=Dict()

    for (g,gen) in data["renewable_generators"]
        for t in 1:T
            rgen_min[g,t]=gen["power_output_minimum"][t]
            rgen_max[g,t]=gen["power_output_maximum"][t]
        end
    end

    # Define the set which contains all of generators
    total_gens = union(thermal_gens, renewable_gens)
    
    # Create the model
    m = Model()

    # Declare all the decision variables
    @variable(m, x[thermal_gens, 1:T], Bin)
    @variable(m, p[total_gens, 1:T])

    # Define the objective
    @objective(m, Min, sum(unit_cost[g]*p[g,t] for g in thermal_gens, t in 1:T))
    
    # Write all the constraints
    @constraint(m, ther_power_const_lhs[g in thermal_gens, t in 1:T], min_power[g]*x[g,t] <= p[g,t])
    @constraint(m, ther_power_const_rhs[g in thermal_gens, t in 1:T], p[g,t] <= max_power[g]*x[g,t])
 
    @constraint(m, demand[t in 1:T], sum(p[g, t] for g in total_gens) >= demand[t])

    @constraint(m, ren_power_const_lhs[g in renewable_gens, t in 1:T], rgen_min[g,t] <= p[g,t])
    @constraint(m, ren_power_const_rhs[g in renewable_gens, t in 1:T], p[g,t] <= rgen_max[g,t])

    # If option 1 is selected, using sum to write constraints
    if option == 1
        @constraint(m, min_up_time_0[g in thermal_gens], 
            sum(x[g,s] for s in 1:min(min_up_time[g],T)) >= (min(min_up_time[g],T))*(x[g,1]))
        @constraint(m, min_up_time[g in thermal_gens, t in 2:T], 
            sum(x[g,s] for s in t:min(t+min_up_time[g]-1,T)) >= (min(t + min_up_time[g]-1,T) - t + 1)*(x[g,t]-x[g,t-1]))

        @constraint(m, min_down_time_0[g in thermal_gens], 
            sum(x[g,s] for s in 1:min(min_down_time[g],T)) <= min_down_time[g]*(1+x[g,1]))
        @constraint(m, min_down_time[g in thermal_gens, t in 2:T], 
            sum(x[g,s] for s in t:min(t+min_down_time[g]-1,T)) <= min_down_time[g]*(1-x[g,t-1]+x[g,t]))
    
    # If option 2 is selected, just write down each of constraints    
    elseif option == 2
        @constraint(m, min_up_time_0[g in thermal_gens, s in 1:min(min_up_time[g],T)], x[g,s] >= x[g,1])
        @constraint(m, min_up_time[g in thermal_gens, t in 2:T, s in t:min(t+min_up_time[g]-1,T)], x[g,s] >= x[g,t] - x[g,t-1])

        @constraint(m, min_down_time_0[g in thermal_gens, s in 1:min(min_down_time[g],T)], x[g,s] <= 1 + x[g,1])           
        @constraint(m, min_down_time[g in thermal_gens, t in 2:T, s in t:min(t+min_down_time[g]-1,T)], x[g,s] <= 1 - x[g,t-1] + x[g,t])

    # If option 3 is selected, using turn on/off variables to represent constraints   
    elseif option == 3
        @variable(m, w[thermal_gens, 1:T], Bin)
        @variable(m, v[thermal_gens, 1:T], Bin)

        @constraint(m, turn_on_off_0[g in thermal_gens], x[g,1] == w[g,1] - v[g,1])
        @constraint(m, turn_on_off[g in thermal_gens, t in 2:T], x[g,t] == x[g,t-1] + w[g,t] - v[g,t])
        
        @constraint(m, min_up_time[g in thermal_gens, t in 1:T], sum(w[g,s] for s in max(t-min_up_time[g]+1,1):t) <= x[g,t])
        @constraint(m, min_down_time[g in thermal_gens, t in 1:T], sum(v[g,s] for s in max(t-min_down_time[g]+1,1):t) <= 1 - x[g,t])      
    end
            
    # Set optimizer, time limit and solve it
    set_optimizer(m, HiGHS.Optimizer) 
    set_time_limit_sec(m,200)
    optimize!(m)

    ## Print required results
    println("---------------------------------------")

    # Print Solution time
    println("Solution time: ", solve_time(m))

    # Print Number of nodes
    println("Number of nodes: ", node_count(m))

    # Print Optimal value
    println("Optimal value: ", objective_value(m))
end    

solve_uc (generic function with 1 method)

In [2]:
### Solve first instance using option 1
solve_uc("2020-12-23.json",1)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
12729 rows, 7008 cols, 53012 nonzeros
11028 rows, 7008 cols, 44069 nonzeros

Solving MIP model with:
   11028 rows
   7008 cols (3504 binary, 0 integer, 0 implied int., 3504 continuous)
   44069 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.3s
         0       0         0   0.00%   135380283.0803  inf                  inf        0      0      2      2784     0.6s
 C       0       0         0   0.00%   137721671.7733  179164726.08      23.13%     2103    229    143      3979     5.2s
         0       0         0   0.00%   141164855.1631  179164726.08      21.21%     6477    731  

In [3]:
### Solve first instance using option 2
solve_uc("2020-12-23.json",2)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
31263 rows, 7008 cols, 89829 nonzeros
21237 rows, 7008 cols, 59751 nonzeros
21237 rows, 7008 cols, 59751 nonzeros

Solving MIP model with:
   21237 rows
   7008 cols (3504 binary, 0 integer, 0 implied int., 3504 continuous)
   59751 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.4s
         0       0         0   0.00%   137218038.8561  inf                  inf        0      0     13      3595     1.1s
 R       0       0         0   0.00%   140191960.793   145539961.2265     3.67%     2395    131     48      4857     3.4s
         0       0         0   0.00%   141108037.5325  1455

In [4]:
### Solve first instance using option 3
solve_uc("2020-12-23.json",3)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
17568 rows, 14016 cols, 69998 nonzeros
15136 rows, 11631 cols, 73550 nonzeros
14843 rows, 11573 cols, 71800 nonzeros

Solving MIP model with:
   14843 rows
   11573 cols (8069 binary, 0 integer, 0 implied int., 3504 continuous)
   71800 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     1.0s
         0       0         0   0.00%   141925304.9536  inf                  inf        0      0     23      2683     1.5s
 R       0       0         0   0.00%   142096840.2122  143221389.8902     0.79%      898     82     86      3197     3.2s
         0       0         0   0.00%   142483874.7611  

In [5]:
### Solve second instance using option 1
solve_uc("2015-08-01_hw.json",1)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
162477 rows, 86448 cols, 849111 nonzeros
140588 rows, 77856 cols, 731517 nonzeros

Solving MIP model with:
   140588 rows
   77856 cols (39504 binary, 0 integer, 0 implied int., 38352 continuous)
   731517 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     3.9s
         0       0         0   0.00%   6597676151.162  inf                  inf        0      0      2     22707     8.9s
         0       0         0   0.00%   6617942993.207  inf                  inf     4326    598    234     27168    19.2s
         0       0         0   0.00%   6631408238.264  inf                  inf     53

In [6]:
### Solve second instance using option 2
solve_uc("2015-08-01_hw.json",2)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
572062 rows, 86448 cols, 1661794 nonzeros
362385 rows, 78912 cols, 1041451 nonzeros
361329 rows, 77856 cols, 1039339 nonzeros

Solving MIP model with:
   361329 rows
   77856 cols (39504 binary, 0 integer, 0 implied int., 38352 continuous)
   1039339 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     7.2s
         0       0         0   0.00%   6667547767.363  inf                  inf        0      0      2     29425    23.8s
         0       0         0   0.00%   6694905243.04   inf                  inf     3158     88     46     30454    35.0s
         0       0         0   0.00%   67

In [7]:
### Solve second instance using option 3
solve_uc("2015-08-01_hw.json",3)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
227328 rows, 187776 cols, 1087193 nonzeros
194626 rows, 156136 cols, 1109490 nonzeros
175126 rows, 144825 cols, 884541 nonzeros

Solving MIP model with:
   175126 rows
   144825 cols (103160 binary, 0 integer, 1976 implied int., 39689 continuous)
   884541 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -81102355.98391 inf                  inf        0      0      0         0    40.3s
         0       0         0   0.00%   6789531657.732  inf                  inf        0      0      2     27379    52.2s
         0       0         0   0.00%   6789541068.401  inf                  inf      208     10      2     28422    61.3s
 R       0       0         0   0.00

In [8]:
### Solve third instance using option 1
solve_uc("2015-01-01_lw.json",1)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
164592 rows, 85776 cols, 868437 nonzeros
147149 rows, 81360 cols, 757182 nonzeros

Solving MIP model with:
   147149 rows
   81360 cols (40944 binary, 0 integer, 0 implied int., 40416 continuous)
   757182 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     4.0s
         0       0         0   0.00%   17330439471.23  inf                  inf        0      0      2     23795     8.3s
         0       0         0   0.00%   17338512717.28  inf                  inf     3871    316      6     26467    18.1s
         0       0         0   0.00%   17343384238.98  inf                  inf     54

In [9]:
### Solve third instance using option 2
solve_uc("2015-01-01_lw.json",2)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
584981 rows, 85776 cols, 1702537 nonzeros
373771 rows, 81888 cols, 1073323 nonzeros
373243 rows, 81360 cols, 1072267 nonzeros

Solving MIP model with:
   373243 rows
   81360 cols (40944 binary, 0 integer, 0 implied int., 40416 continuous)
   1072267 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     7.5s
         0       0         0   0.00%   17347613378.09  inf                  inf        0      0      1     30902    25.2s
         0       0         0   0.00%   17352295839.75  inf                  inf     2363    183      6     32722    40.1s
         0       0         0   0.00%   17

In [10]:
### Solve third instance using option 3
solve_uc("2015-01-01_lw.json",3)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
220224 rows, 179328 cols, 1080556 nonzeros
189110 rows, 149684 cols, 1160021 nonzeros
173430 rows, 140511 cols, 939601 nonzeros

Solving MIP model with:
   173430 rows
   140511 cols (98798 binary, 0 integer, 1086 implied int., 40627 continuous)
   939601 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -17513498.99674 inf                  inf        0      0      0         0    67.7s
         0       0         0   0.00%   17357859158.25  inf                  inf        0      0    298     21804    78.1s
         0       0         0   0.00%   17357925159.62  inf                  inf      378     18    302     24240    97.6s
         0       0         0   0.00%

In [11]:
### Q2.
### Load packages and preprocess data using professor's Robust_knap Starter

## Load packages
using HiGHS, JuMP

## Define function for solving Robust knapsack proplem
function solve_knap(data_file)
    
    # Preprocess data
    filename = data_file
    strdata=split(read(filename, String))
    intdata = [parse(Int,s) for s in strdata]
    nitems=intdata[1]
    items = 1:nitems
    b=intdata[2]
    a=zeros(Int64,nitems)
    c=zeros(Int64,nitems)
    for i in items
        c[i]=intdata[2*i+1]
        a[i]=intdata[2*i+2]
    end
    
    # Assume the maximum deviation is 20% of coefficient
    delta = zeros(Int64,nitems)
    [delta[i] = Int(round(a[i]*0.2)) for i in items]

    # Allow at most 10% of items to deviate to their max simultaneously
    maxnumdevs = Int(round(0.1*nitems))

    # Create the model
    m_knap = Model()
    
    # Declare all the decision variables
    @variable(m_knap, 0 <= x[items] <= 1, Bin)

    @variable(m_knap, z)
    @variable(m_knap, w[items] >= 0)

    # Define the objective
    @objective(m_knap, Max, sum(c[i]x[i] for i in items))

    # Write all the constraints
    @constraint(m_knap, R_XF_1, sum(a[i]x[i] for i in items) + maxnumdevs*z + sum(w[i] for i in items) <= b)
    @constraint(m_knap, R_XF_2[i in items], z + w[i] >= delta[i]x[i])

    # Set optimizer and solve it
    set_optimizer(m_knap, HiGHS.Optimizer)
    optimize!(m_knap)

    # Print required results
    println("---------------------------------------")

    # Print Solution time
    println("Solution time: ", solve_time(m_knap))

    # Print Number of nodes
    println("Number of nodes: ", node_count(m_knap))

    # Print Optimal value
    println("Optimal value: ", objective_value(m_knap))
end

solve_knap (generic function with 1 method)

In [12]:
### Solve first instance
solve_knap("knapPI_2_100_1000_1")

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
101 rows, 201 cols, 501 nonzeros
101 rows, 201 cols, 501 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   101 rows
   201 cols (100 binary, 0 integer, 0 implied int., 101 continuous)
   501 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   51114           -inf                 inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   51114           1276            3905.80%        0      0      0         0     0.0s
 R       0       0         0   0.00%   1340.59375      1277               4.98%        0      0      0        34     0.0s

63.0% inactive integer columns, restarting
Model after restart has 10

In [13]:
### Solve second instance
solve_knap("knapPI_2_200_1000_1")

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
201 rows, 401 cols, 1001 nonzeros
201 rows, 401 cols, 1001 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   201 rows
   401 cols (200 binary, 0 integer, 0 implied int., 201 continuous)
   1001 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   103002          -inf                 inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   103002          1208            8426.66%        0      0      0         0     0.0s

31.5% inactive integer columns, restarting
Model after restart has 201 rows, 338 cols (137 bin., 0 int., 0 impl., 201 cont.), and 875 nonzeros

         0       0         0   0.00%   1407.

In [14]:
### Solve third instance
solve_knap("knapPI_2_1000_1000_1")

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
1001 rows, 2001 cols, 5000 nonzeros
1001 rows, 2001 cols, 5000 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   1001 rows
   2001 cols (1000 binary, 0 integer, 0 implied int., 1001 continuous)
   5000 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   507950          -inf                 inf        0      0      0         0     0.1s
 S       0       0         0   0.00%   507950          7711            6487.34%        0      0      0         0     0.1s

33.7% inactive integer columns, restarting
Model after restart has 1001 rows, 1664 cols (663 bin., 0 int., 0 impl., 1001 cont.), and 4326 nonzeros

         0       0         0   0